# Introducción

Desde su lanzamiento en Enero de 2008 [pandas](https://pandas.pydata.org/) ha sido la librería de análisis tabular más utilizada dentro del ecosistema Python.

Pandas es una librería muy versátil y con mucho soporte por parte de la comunidad de Data Science.

[scikit-learn](https://scikit-learn.org/stable/), [xgboost](https://xgboost.readthedocs.io/en/stable/) y otras muchas herramientas son compatibles con pandas lo que aumenta su popularidad [(~47k estrellas en GitHub)](https://github.com/pandas-dev/pandas) y también extiende su funcionalidad. **No obstante pandas nació con algunas limitaciones importantes que complican su uso para hacer análisis de datasets masivos (aquellos que no caben en la RAM de tu ordenador).**

En los últimos años la librería [polars](https://pola.rs/) ha ganado mucha notoriedad [(~36k estrellas en GitHub)](https://github.com/pola-rs/polars).

Dentro del workshop de hoy vamos a hacer una **breve introducción a polars y lo vamos a comparar con pandas.**

El dataset utilizado para el análisis es [New Yort Taxi and Limousine Comission Tip Dataset](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page).

Para descargar el dataset, podéis ejecutar el código del script `download_mp.py` y para limpiar los datasets podéis utilizar el notebook [Getting_and_Cleaning_Data.ipynb](./Getting_and_Cleaning_Data.ipynb)

# Los "pecados capitales" de pandas.

El creador de pandas Wes McKinney escribió en detalle sobre algunas de las limitaciones que tiene pandas.

Os recomiendo que lean su blog: [Apache Arrow and the “10 Things I Hate About pandas”](https://wesmckinney.com/blog/apache-arrow-pandas-internals/).

Básicamente los principales problemas de pandas según su creador son:

1. **Internals too far from “the metal”.**<br>
1. No support for memory-mapped datasets.<br>
1. Poor performance in database and file ingest/export.<br>
1. Warty missing data support.<br>
1. **Lack of transparency into memory use, RAM management.**<br>
1. Weak support for categorical data.<br>
1. **Complex groupby operations awkward and slow.**<br>
1. Appending data to a DataFrame tedious and very costly.<br>
1. Limited, non-extensible type metadata.<br>
1. **Eager evaluation model, no query planning.**<br>
1. **“Slow”, limited multicore algorithms for large datasets.**<br>

Dentro de este workshop vamos a hablar un poco más en detalle sobre los 5 puntos marcados en **negrita.**

***Nota: el blog anterior fue escrito antes de la aparición de polars en 2020, pero curiosamente polars resuelve algunas de las problemas que se mencionan en el blog.***

---

El punto número 1, **Internals too far from “the metal** se puede resumir en: pandas está escrito en Python y usa como backend numpy (recientemente pandas también ha añadido la posibilidad de usar Arrow como backend).

Esto hace que cuando tenemos un DataFrame **mixto** (strings, números o hasta objetos python en el DataFrame) las operaciones son mucho más lentas.

`In pandas, an array of strings is an array of PyObject pointers, and the actual string data lives inside PyBytes or PyUnicode structs that live all over the process heap.`

---

Pandas consume mucha memoria interna porque en determinas situaciones debe hacer una copia del DataFrame. Si tenemos muchas columnas esto es algo muy ineficiente.

Según el propio Wes McKinney, `pandas rule of thumb: have 5 to 10 times as much RAM as the size of your dataset`.

### Es decir, si nuestro dataset en memoria ocupa 10GB, es recomendable tener entre 50-100GB de RAM para poder trabajar con pandas sin sustos.

---

**Los puntos 7 y 11** están muy relacionados y tienen que ver con que pandas "carga" todo el dataset en memoria para hacer sus cálculos. Lo anterior junto con el **GIL (Global Interpreter Lock)** de Python hacen muy díficil paralelizar operaciones para ganar velocidad por un lado y procesar datasets que no caben en la RAM por el otro lado.

---

Por último pandas únicamente dispone de **Eager mode**. Esto quiere decir que el código se ejecuta según se encuentra. Al no tener un "motor" de ejecución, no hay espacio para optimizar el código (query planner).

# ¿Que es Polars?

Polars es una librería que nació en 2020 y creada por [Ritchie Vink](https://www.ritchievink.com/blog/2021/02/28/i-wrote-one-of-the-fastest-dataframe-libraries/).

Algunas de las diferencias que tiene respecto a pandas son:

1. Está escrita en **Rust**, un lenguaje de programación muy rápido y uno de los más querido según el [stackoverflow survery de 2023](https://survey.stackoverflow.co/2023/).

2. Polars utiliza desde el inicio **Apache Arrow** como forma de representar los datos. Esto hace que el acceso a los datos sea muy rápido y eficiente.

3. Rust permite escribir **código que se ejecuta en paralelo** de manera muy fácil y polars se beneficia enormemente de ello. La mayoría de las operaciones en polars se ejecutan en diferentes threads y se gana mucha velocidad.

4. Polars es según su creador **"un motor de queries con una interfaz de DataFrames"**. Esto implica que tiene dos modos de ejecutar el código: [Eager y Lazy mode](https://docs.pola.rs/user-guide/concepts/lazy-vs-eager/).

**Eager mode al igual que en pandas: el código se ejecuta según se encuentra. No se realiza ninguna optimización de código.**

**Lazy mode por el otro lado permite al motor de ejecución hacer diferentes optimizaciones sobre el código antes de ejecutar la query y por este motivo puede llegar a ser mucho más rápido y eficiente.**

Resumen de las dos herramientas:

|Feature   |Polars   |Pandas   |
|---|---|---|
|Lenguaje   |Rust   |Python   |
|Backend   |Arrow   |Numpy o PyArrow|
|Velocidad   |Muy rápido gracias a Rust, Multicore Compute y Query Planner   |Más rápido que Python puro   |
|Gestión Memoria|Muy eficiente por Rust y cero copias|Copy on Write|
|Lazy Mode|Soportado|No|
|Multicore|Sí by default|No. Tengo que utilizar otras librerías (p.e: Dask)|
|Datasets Masivos|Sí by default|No. Tengo que utilizar otras librerías (p.e: Dask)|
|Facilidad de uso|Intermedio (aprender nueva síntaxis)|Fácil (ya están familiarizados con la herramienta)|
|Comunidad y soporte|Librería nueva con creciente comunidad|Mucha documentación y ejemplos en internet|
|Ciclos de desarrollo|Rápidos|Largos|

# Imports

In [1]:
# ! pip install hvplot

In [2]:
import warnings

warnings.filterwarnings("ignore")

import os


import pandas as pd
import polars as pl

# para hacer los plots con polars
import hvplot.polars

pd.options.display.float_format = "{:.3f}".format
pl.Config.set_float_precision(3);

In [3]:
print(pl.__version__)

0.20.26


In [4]:
print(hvplot.__version__)

0.10.0


In [5]:
# from google.colab import drive
# drive.mount('/content/drive')

# GLOBAL_VARS

In [6]:
CWD = os.getcwd()
PATH_INPUT_FOLDER = os.path.join(CWD, "output")

In [7]:
# si estamos en colab
# especificar la ruta donde esta el fichero "yellow_tripdata_2024-01.parquet"
# por ejemplo "/content/drive/MyDrive/POLARS/input"
# PATH_INPUT_FOLDER = ""

In [8]:
TRIP2401 = os.path.join(PATH_INPUT_FOLDER, "yellow_tripdata_2024-01.parquet")

In [9]:
fs = [
    f
    for f in os.listdir(PATH_INPUT_FOLDER)
    if f not in [".ipynb_checkpoints", ".DS_Store"]
]

In [10]:
print(
    f"Tenemos un total de {len(fs)} ficheros dentro de nuestra carpeta\n{PATH_INPUT_FOLDER}"
)

Tenemos un total de 59 ficheros dentro de nuestra carpeta
/Users/nicolaepopescul/code/polanyt/output


# Pandas vs Polars: métodos básicos

Vamos a cargar los dos datasets en memoria para comparar la sintaxís e ir conociendo polars.

Aquí vamos a utilizar la `API de Eager mode de polars` y por este motivo vamos a utilizar el método de `read_parquet` de polars.

Más adelante veremos los `LazyFrames` de polars.

In [11]:
# cargar el dataset
# pandas

In [12]:
# polars

In [13]:
# hacer el shape del dataset
# pandas

In [14]:
# polars

In [15]:
# hacer el describe del dataset
# pandas

In [16]:
# polars

In [17]:
# hacer el info del dataset
# pandas

In [18]:
# polars

In [19]:
# sacar las columnas del dataset
# pandas

In [20]:
# polars

In [21]:
# ver las primeras 5 filas (head)
# pandas

In [22]:
# polars

In [23]:
# sacar una muestra del dataset
# pandas

In [24]:
# polars

# .with_columns, .select, .filter ...

In [25]:
# sacar la media de la column PassengerCount
# pandas

In [26]:
# polars

### `Came for speed, stayed for the syntax.`

Dentro de polars hay una sintaxis específica que se recomienda seguir.

Vamos a ver algunas "peculiaridades de la sintaxis de polars".

* Para seleccionar una columna debemos usar `.select`

En polars al `.select` se le conoce como `contexto` y es el **"entorno"** donde se van a evaluar `expresiones de polars`.

Existen otros contextos dentro de polars. Por ejemplo: `group_by`, `select`, `filter` son todo contextos de polars.

Por el otro lado, podemos pensar en `expresiones` como operaciones que queremos que se evaluen y ejecuten dentro de un contexto. Por ejemplo: `sum`, `min`, `max` etc. son todo expresiones que se pueden evaluar dentro de determinados contextos.

In [27]:
# sacar la media de la column PassengerCount

# usando el select y pl.col
# polars

* Si queremos crear una nueva columna debemos usar `.with_columns`

In [28]:
# calcular el número total de nulos que hay en la columna de PassengerCount

# usar el assign de pandas
# pandas

In [29]:
# polars

Polars distingue entre `nulls` y `nans`.

Los `nulls` son valores `missing` dentro de nuestro dataset.

Los `nans` por el otro lado son `not a number`. Por ejemplo: `infinito`.

In [30]:
# calcular el número total de nulos y nans que hay en la columna de PassengerCount

# filtrar PassengerCount
# usar el assign
# drop PassengerCount
# pandas

In [31]:
# usar with_columns
# select y sum
# polars

In [32]:
# calcular el número total de nulos que hay por columna

# usar with_columns con pl.all()
# melt
# rename
# filter
# polars

* Cuando queremos filtrar un dataframe podemos utilizar el método de `.filter` de polars.

In [33]:
# calcular el min, mean y max por StoreAndFwdFlag cuando VendorId == 1 y no hay nulos en PassengerCount
# filtrar primero
# groupby
# agg
# sort
# reset
# pandas

In [34]:
# filter
# group_by
# agg
# sort
# polars

# .group_by 

In [35]:
# calcular el TotalTripDistance por día de la semana entre 2024-1-1 y 2024-2-1 (inclusive) y cuando
# no hay nulos en PassengerCount

# filter
# assing DayOfWeek
# groupby
# agg
# sort_values
# pandas

In [36]:
# filter
# with_columns pl.col is_between
# with_columns DayOfWeek
# group_by
# agg
# sort
# polars

# Plots

Tanto pandas como polars tienen capacidad para realizar gráficos desde la propia librería.

Pandas utiliza [Matplotlib](https://matplotlib.org/) mientras que polars utiliza [hvPlot](https://hvplot.holoviz.org/).

In [37]:
# hacer un bar plot de la distancia media por VendorId

# groupby
# agg
# reset y sort
# Vendor, MeanTripDistance
# plot
# pandas

In [38]:
# group_by
# agg
# sort
# select pl.col().cast(str)
# hvplot
# polars

# Lazy Mode

Cuando utilizamos cualquier de los métodos de polars que empiezan por `scan_` estamos usando el `Lazy Mode` de polars.

Esto implica que: 
1. No estamos cargando el dataset en memoria.
2. El query planner puede hacer algunas optimizaciones de nuestro código.

In [39]:
# mostrar el pl.scan

In [40]:
# ver el naive query plan

In [41]:
# ver el naive query plan de la siguiente operación

# crear un df con 3 columnas
# TpepPickupDatetime, DayOfWeek, FlagPassengerCount
# cuando TpepPickupDatetime está entre 2024-1-1 y 2024-2-1
# no hay nulos en PassengerCount
# y cuando el PassengerCount es superior a >= 3

# filter
# with DayOfWeek y FlagPassengerCount >= 3
# select

In [42]:
# mostrar el query plan optimizado
# show_graph

In [43]:
# mostrar el método de collect para ejecutar y "recolectar" los resultados
# sacar el total de nulos que hay entre todas las columnas

# select pl.all().is_null()
# sum
# collect

In [44]:
COLS = [
    "VendorID",
    "TpepPickupDatetime",
    "TpepDropoffDatetime",
    "PassengerCount",
    "TripDistance",
    "RatecodeID",
    "StoreAndFwdFlag",
    "PULocationID",
    "DOLocationID",
    "PaymentType",
    "FareAmount",
    "Extra",
    "MtaTax",
    "TipAmount",
    "TollsAmount",
    "ImprovementSurcharge",
    "TotalAmount",
    "CongestionSurcharge",
    "AirportFee",
]

Con LazyFrames de `polars` podemos procesar **194.457.948 de registros.**

In [45]:
# mostrar como podemos procesar nuestro dataset de 194.457.948 de registros

# select pl.col(COLS)
# collect
# shape

El tamaño total de nuestro dataset en memoria es de **26GB**.

In [46]:
# calcular el tamaño de nuestro dataset en memoria

# select pl.col(COLS)
# collect
# estimated_size()

In [47]:
# hacer el head del dataset

In [48]:
# select TpepPickupDatetime y PassengerCount

# nos quedamos sólo con aquellas filas que no tienen ningún nulo a nivel de fila
# ~pl.all_horizontal(pl.all().is_null())

# filter TpepPickupDatetime entre 2020-12-31 y 2024-3-1
# sort TpepPickupDatetime
# group_by_dynamic(index="TpepPickupDatetime", every)

# agg (TotalPassengers)

# collect
# sort
# hvplot

In [49]:
# 2.49 s ± 190 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

In [50]:
# %%timeit

# Conclusión

Tras este breve workshop, podemos extrar algunas conclusiones:
1. Tanto pandas como polars son excelentes librerás para hacer análisis de datos.
2. Pandas es una excelente opción siempre y cuando tenemos datasets pequeños y que caben en memoria.
3. Pandas también es una librería mucho más madura que polars y que tiene mucha documentación y ejemplos.
4. Polars es una de las mejores opciones cuando tenemos que trabajar con dataset grandes que no caben en memoria y no podemos utilizar tecnologías Big Data.
5. Polars está en fase activa de desarrollo.

Recomendación personal: si vas a empezar un nuevo proyecto utiliza polars (aprender la librería etc).

No hagan "grandes migraciones de código" porque es muy costoso a nivel de tiempo.

Si caso de echar en falta algo de pandas, siempre pueden utilizar `.to_pandas()`

In [51]:
# polars to pandas